# This is the new version of Skimpy from github

In [1]:
from skimpy.io.yaml import load_yaml_model, export_to_yaml

met_model = 'ecoli_shikki_regulation'
base = f'.'
path_to_kmodel = f'{base}/kinetic_model_scaffold.yaml'
output_model_path = f'{base}/output/kin_varma_curated_regulation.yaml'
kmodel = load_yaml_model(path_to_kmodel)

2024-03-01 16:20:40,046 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [2]:
from skimpy.io.regulation import load_enzyme_regulation
import pandas as pd
# Add regulation data to kinetic model
path_to_regulation_data = f'{base}/allosteric_regulations.csv'
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel.reactants.keys()))]

print(kmodel.reactions['ACt2rpp'].modifiers)

kmodel = load_enzyme_regulation(kmodel, df_regulations_all)

TabDict([('DSM_h_c', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f7a8fcccb70>), ('DSM_h_p', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f7a8fccc6a0>)])
Added regulation reaction ACALD of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f7a8fa3dac8> kinetics 
Added regulation reaction ADK1 of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f7a8fa3da58> kinetics 
Added regulation reaction ANS of with <skimpy.mechanisms.convenience_with_inihibition.make_convenience_with_inhibition.<locals>.ConvenienceInhibited object at 0x7f7a8fa407b8> kinetics 
Added regulation reaction CHORM of with <skimpy.mechanisms.generalized_reversible_hill_n_n_h1_with_inhibition.make_generalized_reversible_hill_n_n_h1_with_inhibition.<locals>.H1GeneralizedReversibleHillInhibited object at 0x7f7a8fa432e8> 

In [3]:
no_km = 0
no_ka = 0
no_ki = 0
for k,v in kmodel.parameters.items():
    if k.startswith("km_"):
        no_km += 1 
    elif ("activation" in k):
        no_ka += 1
    elif ("inhibition" in k):
        no_ki += 1
print(f"The model has {len(kmodel.reactions)} reactions.")
print(f"The number of km in this model is {no_km}.")
print(f"The number of inhibition k in this model is {no_ki}.")
print(f"The number of activation k in this model is {no_ka}.")


The model has 123 reactions.
The number of km in this model is 384.
The number of inhibition k in this model is 12.
The number of activation k in this model is 10.


In [2]:
export_to_yaml(kmodel, output_model_path)

ValueError: dictionary update sequence element #0 has length 1; 2 is required